<h1><center><font size=10> Generative AI Project</center></font></h1>

# **Financial Product Complaint Classification and Summarization**

## **Business Context**

### **Description**
*In the modern financial industry, customer complaints play a crucial role in identifying areas where financial institutions can improve their services. Effectively categorizing these complaints into specific product categories, such as credit reports, student loans, or money transfers, is essential for addressing customer concerns promptly by routing the tickets to relevant personnel. Leveraging Generative AI for text classification can help financial institutions better understand customer grievances and respond more efficiently. Apart from this, a summary of the customer complaint helps the support personnel quickly grasp the gist of the grievance*

### **Objective**
*The primary goal of this project is to utilize Generative AI techniques to improve the classification and summarization of customer complaints in the financial sector.
Specifically, the project will focus on:*

1. **Text-to-Label Classification:** *Implementing Zero-shot and Few-shot prompting methods to accurately classify customer complaints into relevant product categories.*
2. **Text-to-Text Summarization:** *Using Zero-shot prompting to generate concise summaries of customer complaints, enabling more personalized and effective responses.*

### **Conclusion**
*Upon completing this project, you will have the capability to develop end-to-end applications for LLM-based text classification and summarization. These tools will enable financial institutions to automate the complaint handling process, leading to faster, more accurate responses to customer issues, improved customer satisfaction, and enhanced compliance with industry regulations. This project will also provide you with valuable skills and experience that can be applied to a range of real-world business challenges.*


# **Section 1 : Setting Up for Prompt Engineering with Mistral Model**

### **Install & Importing neccessary libraries**

In [ ]:
!apt-get update
!apt-get install -y ninja-build cmake
!pip install ipywidgets --upgrade

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]                           
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.5 kB]                 
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,234 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]                           
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,619 kB]                      
Get:

In [ ]:
# This part of code will skip all the un-necessary warnings which can occur during the execution of this project.
import warnings
warnings.filterwarnings("ignore", category=Warning)

In [ ]:
# Installation for GPU llama-cpp-python==0.2.69
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69
# For downloading the models from HF Hub
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 37.6 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llama-cpp-python
Failed to build llama-cpp-python
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (llama-cpp-python)


In [ ]:
!pip install evaluate==0.4.3
!pip install bert-score==0.3.13

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00


In [ ]:
!pip freeze > requirement.txt

In [ ]:
# Basic Imports for Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re

import torch
import evaluate

# from google.colab import drive
import locale

### **Question 1: Importing Libaries and Mistral Model (3 Marks)**

- For the Mistral Model name or path and model basename, refer to the **Week 3 Additional Content: Prompt Engineering Fundamentals**
- Code Notebook: Self-Consistency and Tree-of-Thought Prompting with Llama 2 and Mistral.





https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/blob/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf

In [ ]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 MB 26.8 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.6-cp310-cp310-linux_x86_64.whl size=4070564 sha256=ac572a1b37dc0b46a5cc647a136af96dbb357c2ed30539c408079c19d792adaa
  Stored in directory: /root/.cache/pip/wheels/06/61/7b/d5ebacdccd230f42c847b6eec707a26c8455f49277393b295c
Successfully built llama-cpp-python


In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

mistral-7b-instruct-v0.2.Q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

In [ ]:
lcpp_llm = Llama(
        model_path=model_path,
        n_threads=2,  # CPU cores
        n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
        n_ctx=4096,  # Context window
    )

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loa

# **Section 2: Text to Label generation**

### **Question 2: Zero-Shot Prompting for Text Classification (5 Marks)**

##### **Q2.1: Define the Prompt Template, System Message, generate_prompt** **(2 Marks)**

- Define a **system message** as a string and assign it to the variable system_message to generate product class.
- Create a **zero shot prompt template** that incorporates the system message and user input.
- Define **generate_prompt** function that takes both the system_message and user_input as arguments and formats them into a prompt template


Write a Python function called **generate_mistral_response** that takes a single parameter, narrative, which represents the user's complain. Inside the function, you should perform the following tasks:


- **Combine the system_message and narrative to create a prompt string using generate_prompt function.**

*Generate a response from the Mistral model using the lcpp_llm instance with the following parameters:*

- prompt should be the combined prompt string.
- max_tokens should be set to 1200.
- temperature should be set to 0.
- top_p should be set to 0.95.
- repeat_penalty should be set to 1.2.
- top_k should be set to 50.
- stop should be set as a list containing '/s'.
- echo should be set to False.
Extract and return the response text from the generated response.

Don't forget to provide a value for the system_message variable before using it in the function.

In [ ]:
system_message = """
You are a financial assistant tasked to classify customer complaints into relevant product categories.

Customer complaints will be delimited by triple backticks in the input. Classify the customer complaint summary description presented in the input as follows:

'credit_card'
'retail_banking'
'credit_reporting'
'mortgages_and_loans'
'debt_collection'

Answer only with one of the labels above. Do not explain your answer.

Instructions:
1. Read the compliant text carefully and think through the options for product classification
3. Consider the overall customer complaint and estimate the probability of the product category
"""

In [ ]:
zero_shot_prompt_template = """<s>[INST]\n <<SYS>> \n {system_message} \n <</SYS>>```{user_message}``` /n [/INST] """

In [ ]:
def generate_prompt(system_message,user_input):
    prompt=zero_shot_prompt_template.format(system_message=system_message,user_message=user_input)
    return prompt



In [ ]:
def generate_mistral_response(input_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message,input_text)

    # Define the Llama model along with its parameters for generating a response
    response = lcpp_llm(
    prompt=prompt,
    max_tokens=1200,
    temperature=0,
    top_p=0.95,
    top_k=50,
    repeat_penalty=1.2,
    stop=['/s'],
    echo=False # do not return the prompt
)

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    print(response_text)
    return response_text

In [ ]:
# Load a CSV File containing Dataset of 500 products, narrative and summary (summary of narrative)
import pandas as pd
data_path = "/kaggle/input/complains-data/Complains_classification.csv"
data = pd.read_csv(data_path)

In [ ]:
# Get the proportion of each class
proportions = data['product'].value_counts(normalize=True) * 100

# Print the results
print(proportions)

product
credit_reporting       77.6
mortgages_and_loans     7.2
debt_collection         5.8
credit_card             5.6
retail_banking          3.8
Name: proportion, dtype: float64


In [ ]:
# Randomly select 30 rows
new_data = data.sample(n=30, random_state=40)


##### **Q2.2: Create a new column in the DataFrame called 'mistral_response' and populate it with responses generated by applying the 'generate_mistral_response' function to each 'narrative' in the DataFrame and prepare the mistral_response_cleaned column using extract_category function** **(1 Marks)**

In [ ]:
# example - new_data['mistral_response'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response'] = new_data['narrative'].apply(lambda x: generate_mistral_response(x))

llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   389 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    43 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  104421.55 ms /   432 tokens
Llama.generate: 170 prefix-match hit, remaining 509 prompt tokens to eval


 credit_card.

This complaint is about a fraudulent charge made on a debit card associated with Capital One, indicating that the product category for this complaint should be classified as 'credit\_card'.


llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   509 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    50 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  134982.63 ms /   559 tokens
Llama.generate: 170 prefix-match hit, remaining 186 prompt tokens to eval


 credit_reporting

(Note: Based on the provided text, it appears that the customer complaint is related to issues with consumer reporting agencies and identity theft. Therefore, I have classified this as a 'credit_reporting' issue.)


llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   186 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     8 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   44698.96 ms /   194 tokens
Llama.generate: 170 prefix-match hit, remaining 509 prompt tokens to eval


...

 credit_reporting


llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   509 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    50 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  136515.88 ms /   559 tokens
Llama.generate: 170 prefix-match hit, remaining 99 prompt tokens to eval


 credit_reporting

(Note: Based on the provided text, it appears that the customer complaint is related to issues with consumer reporting agencies and identity theft. Therefore, I have classified this as a 'credit_reporting' issue.)


llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    99 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     4 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   23458.40 ms /   103 tokens


 credit_reporting


In [ ]:
new_data['mistral_response']

167     credit_card.\n\nThis complaint is about a fra...
169     credit_reporting\n\n(Note: Based on the provi...
461                  credit_reporting / debt\_collection
253     credit_reporting\n\n(Note: Based on the provi...
42      retail_banking\n\nThis complaint appears to b...
369     credit_reporting\n\n(Note: Based on the provi...
26                                 'mortgages_and_loans'
377     credit_reporting\n\n(Note: Based on the provi...
238     credit_reporting\n\n(Note: Based on the provi...
374     credit_reporting\n\n(Note: Based on the provi...
140     credit_reporting\n\nThis complaint appears to...
175     credit_reporting\n\n(Note: Based on the provi...
388     credit_reporting\n\n(Note: Based on the provi...
62                                 'mortgages_and_loans'
256     credit_reporting\n\n(Note: Based on the provi...
332     credit_reporting\n\n(Note: Based on the provi...
386     credit_reporting\n\n(Note: Based on the provi...
56      credit_card.\n\nThis co

In [ ]:
def extract_category(text):
    # Define the regex pattern to match "category:" or "Category:" followed by a word
    pattern = r'category:\s*(\w+)'  # The pattern itself remains the same

    # Use re.search with the re.IGNORECASE flag to make it case-insensitive
    match = re.search(pattern, text, re.IGNORECASE)

    # If a match is found, return the captured group, else return None
    if match:
        return match.group(1)
    else:
        pattern1 = r'(credit_card|retail_banking|credit_reporting|mortgages_and_loans|debt_collection)'
        match = re.search(pattern1, text, re.IGNORECASE)
        if match:
            return match.group()
        else:
            return ''

In [ ]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response_cleaned'] = new_data['mistral_response'].apply(lambda x: extract_category(x) )

In [ ]:
new_data.tail()

,product,narrative,summary,mistral_response,mistral_response_cleaned
364,credit_reporting,except otherwise provided section consumer rep...,The text specifies the legal procedures and re...,credit_reporting\n\n(Note: Based on the provi...,credit_reporting
139,credit_reporting,true identity theft victim identity theft info...,The input is from an individual who is a victi...,"""credit_reporting""",credit_reporting
150,credit_reporting,true identity theft victim identity theft info...,The input discusses an individual who has disc...,credit_reporting,credit_reporting
173,credit_reporting,block except otherwise provided section consum...,This passage discusses the various provisions ...,credit_reporting\n\n(Note: Based on the provi...,credit_reporting
463,credit_reporting,mother called remove authorized user never use...,The individual's mother requested to remove he...,credit_reporting,credit_reporting


##### **Q2.3: Calculate the F1 score** **(1 Marks)**

In [ ]:
# Calculate F1 score for 'product' and 'mistral_response'
f1 =  f1_score(new_data['product'], new_data['mistral_response'],average='micro')

print(f'F1 Score: {f1}')

F1 Score: 0.0


In [ ]:
# Calculate F1 score for 'product' and 'mistral_response_cleaned'
f2 =  f1_score(new_data['product'], new_data['mistral_response_cleaned'], average='micro')
print(f'F1 Score: {f2}')

F1 Score: 0.8333333333333334


##### **Q2.4: Explain the difference in F1 scores between mistral_response and mistral_response_cleaned.** **(1 Marks)**


The mistral_response contains a label followed by an explanatory text, which describes the reasoning behind the model's classification. However, this explanation often leads to a mismatch between the actual class in the product variable and the extracted class in mistral_response, making it difficult for the model to compute an accurate F1 score. To address this, the mistral_response was cleaned by removing all text except the initial label, which corresponds to the class the model is supposed to predict. After cleaning, the F1 score improved significantly, as the extracted class now better aligns with the product category. For instance, a complaint related to credit reporting was initially classified with additional explanation, resulting in a mismatch and an F1 score of 0. However, after removing the extra text, 43 values correctly matched the product class, leading to a much improved F1 score of 83.

### **Question 3: Few-Shot Prompting for Text Classification (7 Marks)**

##### **Q3.1: Prepare examples for a few-shot prompt, formulate the prompt, and generate the Mistral response. (5 Marks)**

**Generate a set of gold examples by randomly selecting 10 instances of user_input and assistant_output from dataset ensuring a balanced representation with 2 examples from each class.**

In [ ]:
import json
review_1 = data[data['product'] == 'credit_card']
review_2 = data[data['product'] == 'retail_banking']
review_3 = data[data['product'] == 'credit_reporting']
review_4 = data[data['product'] == 'mortgages_and_loans']
review_5 = data[data['product'] == 'debt_collection']

# Sample 2 examples for each category
examples_1 = review_1.sample(2, random_state=40)
examples_2 = review_2.sample(2, random_state=40)
examples_3 = review_3.sample(2, random_state=40)
examples_4 = review_4.sample(2, random_state=40)
examples_5 = review_5.sample(2, random_state=40)

# Concatenate examples for few shot prompting
examples_df = pd.concat([examples_1,examples_2,examples_3,examples_4,examples_5 ])

# Create the training set by excluding examples
gold_examples_df = data.drop(index=examples_df.index)

# Convert examples to JSON
columns_to_select = ['narrative', 'product']
examples_json = examples_df[columns_to_select].to_json(orient='records')

# Print the first record from the JSON
print(json.loads(examples_json)[0])

# Print the shapes of the datasets
print("Examples Set Shape:", examples_df.shape)
print("Gold Examples Shape:", gold_examples_df.shape)

{'narrative': 'called request new york state covid relief plan day interest fee waived amex provided relief leading late payment amex refused honor relief day gap insists charging late fee', 'product': 'credit_card'}
Examples Set Shape: (10, 3)
Gold Examples Shape: (490, 3)


- Define your **system_message**.
- Define **first_turn_template**, **example_template** and **prediction template**
- **create few shot prompt** using gold examples and system_message
- Randomly select 50 rows from test_df as test_data
- Create **mistral_response** with **mistral_response_cleaned** columns for this

In [ ]:
system_message = """
You are a financial assistant tasked with classifying customer complaints into one of the following product categories:

- 'credit_card': Issues related to credit card usage, charges, payments, or fraud.
- 'retail_banking': Problems with checking accounts, savings accounts, or transfers.
- 'credit_reporting': Disputes or concerns about credit history, reporting inaccuracies, or fraud alerts.
- 'mortgages_and_loans': Complaints about home loans, auto loans, or other lending products.
- 'debt_collection': Concerns about debt recovery practices or related disputes.

**Guidelines**:
1. Customer complaints will be delimited by triple backticks (` ``` `) in the input.
2. Carefully analyze the text for keywords, context, and intent.
3. Match the complaint to the most relevant category based on the primary issue described.
4. Answer with only one label from the list above. Do not include any explanation or additional text.

**Examples**:
Input: ```inquired tx auto loan inquiry scheduled continue record```
Output: 'mortgages_and_loans'

Input: ```inquired oh charge card inquiry scheduled continue record```
Output: 'credit_card'

Input: ```fraudulent application submitted name identity used without consent fraudulently obtain good service extend credit without first contacting personally verifying application information day evening```
Output: 'credit_reporting'

Input: ```inquired fl ca real estate inquiry scheduled continue record```
Output: 'mortgages_and_loans'

Input: ```inquired ga unspecified inquiry scheduled continue record```
Output: 'credit_reporting'

Instructions:
1. Read the compliant text carefully and think through the options for product classification
2. Consider the overall customer complaint and estimate the probability of the product category
"""

In [ ]:
first_turn_template = """<s>[INST]\n <<SYS>> \n {system_message} \n <</SYS>>```{user_message}``` \n [/INST] \n{assistant_message}\n</s> """

examples_template = """<s>[INST]\n ```{user_message}``` \n [/INST] \n {assistant_message}\n</s>"""

prediction_template = """<s>[INST]\n ```{user_message}```[/INST]"""

In [ ]:
def create_few_shot_prompt(system_message, examples_df):

    """
    Return a prompt message in the format expected by Mistral 7b.
    10 examples are selected randomly as golden examples to form the
    few-shot prompt.
    We then loop through each example and parse the narrative as the user message
    and the product as the assistant message.

    Args:
        system_message (str): system message with instructions for classification
        examples(DataFrame): A DataFrame with examples (product + narrative + summary)
        to form the few-shot prompt.

    Output:
        few_shot_prompt (str): A prompt string in the Mistral format
    """

    few_shot_prompt = ''

    columns_to_select = ['narrative', 'product']
    examples = (
        examples_df.loc[:, columns_to_select].to_json(orient='records')
    )

    for idx, example in enumerate(json.loads(examples)):
        user_input_example = example['narrative']
        assistant_output_example = example['product']

        if idx == 0:
            few_shot_prompt += first_turn_template.format(
                system_message=system_message,
                user_message=user_input_example,
                assistant_message=assistant_output_example
            )
        else:
            few_shot_prompt += examples_template.format(
                user_message=user_input_example,
                assistant_message=assistant_output_example
            )

    return few_shot_prompt

In [ ]:
few_shot_prompt = create_few_shot_prompt(system_message,examples_df)

In [ ]:
print(few_shot_prompt)

<s>[INST]
 <<SYS>> 
 
You are a financial assistant tasked with classifying customer complaints into one of the following product categories:

- 'credit_card': Issues related to credit card usage, charges, payments, or fraud.
- 'retail_banking': Problems with checking accounts, savings accounts, or transfers.
- 'credit_reporting': Disputes or concerns about credit history, reporting inaccuracies, or fraud alerts.
- 'mortgages_and_loans': Complaints about home loans, auto loans, or other lending products.
- 'debt_collection': Concerns about debt recovery practices or related disputes.

**Guidelines**:
1. Customer complaints will be delimited by triple backticks (` ``` `) in the input.
2. Carefully analyze the text for keywords, context, and intent.
3. Match the complaint to the most relevant category based on the primary issue described.
4. Answer with only one label from the list above. Do not include any explanation or additional text.

**Examples**:
Input: ```inquired tx auto loan in

In [ ]:
def generate_prompt(few_shot_prompt,new_review):
    prompt =  few_shot_prompt + prediction_template.format(user_message=new_review)
    return prompt

In [ ]:
def generate_mistral_response(support_ticket_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message,support_ticket_text)

    # Define the Llama model along with its parameters for generating a response
    response = lcpp_llm(
    prompt=prompt,
    max_tokens=1200,
    temperature=0,
    top_p=0.95,
    top_k=50,
    repeat_penalty=1.2,
    stop=['/s'],
    echo=False) # do not return the prompt


    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    print(response_text)
    return response_text

In [ ]:
# Randomly select 50 rows from gold_examples
new_data = gold_examples_df.sample(n=50, random_state=40)

In [ ]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response'] = new_data['narrative'].apply(lambda x: generate_mistral_response(x))

Llama.generate: 1 prefix-match hit, remaining 1109 prompt tokens to eval
llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  1109 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     4 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  253344.29 ms /  1113 tokens
Llama.generate: 452 prefix-match hit, remaining 433 prompt tokens to eval


 credit\_reporting


llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   433 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    36 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  113276.79 ms /   469 tokens
Llama.generate: 452 prefix-match hit, remaining 18 prompt tokens to eval


 This text appears to be a section of legislation or regulation related to consumer reporting agencies and identity theft. It does not contain a customer complaint, so no product category label is applicable.


llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    18 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     9 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    7634.63 ms /    27 tokens
Llama.generate: 452 prefix-match hit, remaining 421 prompt tokens to eval


...

 'mortgages_and_loans'


llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   421 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    38 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  111358.03 ms /   459 tokens
Llama.generate: 452 prefix-match hit, remaining 433 prompt tokens to eval


 This text appears to be a section of legislation or regulation related to identity theft and consumer reporting agencies, rather than a customer complaint. Therefore, no product category label is applicable in this case.


llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   433 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    36 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  113525.24 ms /   469 tokens


 This text appears to be a section of legislation or regulation related to consumer reporting agencies and identity theft. It does not contain a customer complaint, so no product category label is applicable.


In [ ]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response_cleaned'] = new_data['mistral_response'].apply(lambda x: extract_category(x))

##### **Q3.2: Calculate the F1 score** **(1 Marks)**

In [ ]:
# Calculate F1 score for 'product' and 'mistral_response'
f1 =  f1_score(new_data['product'], new_data['mistral_response_cleaned'], average='micro')

print(f'F1 Score: {f1}')

F1 Score: 0.52


In [ ]:
# Calculate F1 score for 'product' and 'mistral_response'
f1 =  f1_score(new_data['product'], new_data['mistral_response_cleaned'], average='micro')

print(f'F1 Score: {f1}')

F1 Score: 0.52


##### **Q3.3: Share your observations on the few-shot and zero-shot prompt techniques. (1 Marks)**


Few-shot prompting uses a few examples to help the model learn new tasks quickly, while zero-shot prompting relies entirely on the model's pre-trained knowledge to generate responses. In this case, the F1 scores for both few-shot and zero-shot prompting are quite similar, with an F1 score of 0.52 for both techniques. This suggests that the task of classifying customer complaints into predefined categories is relatively simple, and the model's pre-trained knowledge is sufficient to handle it without the need for additional examples. While few-shot prompting could potentially improve performance by providing specific examples, the difference in F1 scores is minimal, indicating that zero-shot prompting performs just as well for this straightforward classification task. The complaints are not overly complex, and the model’s existing knowledge, which is relevant to tasks like sentiment analysis with general categories, seems to be adequate for routing the complaints to the appropriate department.

# **Section 3: Text to Text generation**

### **Question 4: Zero-Shot Prompting for Text Summarization (5 Marks)**

##### **Q4.1: Define the Prompt Template, System Message, generate prompt and model response** **(2 Marks)**


- Define a **system message** as a string and assign it to the variable system_message to generate summary of narrative in data.
- Create a **zero shot prompt template** that incorporates the system message and user input.
- Define **generate_prompt** function that takes both the system_message and user_input as arguments and formats them into a prompt template


Write a Python function called **generate_mistral_response** that takes a single parameter, narrative, which represents the user's complain. Inside the function, you should perform the following tasks:


- **Combine the system_message and narrative to create a prompt string using generate_prompt function.**

*Generate a response from the Mistral model using the lcpp_llm instance with the following parameters:*

- prompt should be the combined prompt string.
- max_tokens should be set to 1200.
- temperature should be set to 0.
- top_p should be set to 0.95.
- repeat_penalty should be set to 1.2.
- top_k should be set to 50.
- stop should be set as a list containing '/s'.
- echo should be set to False.
Extract and return the response text from the generated response.

Don't forget to provide a value for the system_message variable before using it in the function.

In [ ]:
system_message = """
Summarize the customer compliant mentioned in the user input below. Be specific and concise in your summary.
The dialogue will be delimited by triple backticks, that is, ```.
"""

In [ ]:

zero_shot_prompt_template = """<s>[INST]\n <<SYS>> \n {system_message} \n <</SYS>>```{user_message}``` /n [/INST] """

def generate_prompt(system_message,user_input):
    prompt=zero_shot_prompt_template.format(system_message=system_message,user_message=user_input)
    return prompt

def generate_mistral_response(input_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message,input_text)

    # Define the Llama model along with its parameters for generating a response
    response = lcpp_llm(
    prompt=prompt,
    max_tokens=1200,
    temperature=0,
    top_p=0.95,
    top_k=50,
    repeat_penalty=1.2,
    stop=['/s'],
    echo=False) # do not return the prompt


    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    print(response_text)
    return response_text

##### **Q4.2: Generate mistral_response column containing LLM generated summaries** **(1 Marks)**

In [ ]:
# Randomly select 50 rows
gold_examples = data.sample(n=50 , random_state=40)

In [ ]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
gold_examples['mistral_response'] = gold_examples['narrative'].apply(lambda x: generate_mistral_response(x))

Llama.generate: 1 prefix-match hit, remaining 282 prompt tokens to eval
llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   282 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   202 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  143461.64 ms /   484 tokens
Llama.generate: 64 prefix-match hit, remaining 509 prompt tokens to eval


The customer reported a fraudulent charge of an unknown amount on their Capital One checking account made via debit card. The charge was immediately canceled, and the customer disputed it with Capital One. A provisional credit was posted to the account pending determination of the claim. However, the bank denied the claim after receiving a form letter. The customer believes that their debit card was intercepted and fraudulently activated, as they never lost possession of it. They were issued a new debit card via USPS but believe that the original card was used to make unauthorized purchases. The customer has complained about this issue multiple times and received several denials from Capital One's determination person. They feel that there is malfeasance on the part of Capital One for sending a replacement card without authorization and allowing it to be activated and used for credit purchases without their consent. The customer reported the fraudulent charge to both Capital One and th

llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   509 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   244 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  211968.18 ms /   753 tokens
Llama.generate: 64 prefix-match hit, remaining 186 prompt tokens to eval


 The customer complaint is about a consumer reporting agency's handling of identity theft reports. The Consumer Reporting Agency (CRA) is required to block reporting information if a consumer identifies potential identity theft and provides appropriate proof. Once the CRA receives such a report, they must promptly notify any business that has reported information relating to the transaction in question.

The CRA may decline to block or rescind the block if it determines there was no error or material misrepresentation. The consumer shall be notified promptly and in writing of any reinstatement of blocked information.

If a reseller file contains disputed information, the CRA is required to apply certain provisions regarding verification companies and check service companies. A check service company must report identity theft information to the national CRA, but they cannot access blocked information without proper authorization or verification. The consumer's ability to obtain credit m

llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   186 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   164 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  106690.75 ms /   350 tokens
Llama.generate: 64 prefix-match hit, remaining 509 prompt tokens to eval


...

 A customer expressed concern about Fidelity Capital Holding Inc. acquiring their consumer credit card debt and notifying them of a court date for a levied asset, specifically on checking and saving accounts. The levy occurred yesterday, leaving the account holder with insufficient funds to pay mortgage payments, non-mortgage expenses, utilities, HOA fees, gas, food, and ineligible public assistance. As a middle wage earner, they are struggling to make ends meet and find it unjust that Fidelity purchased this debt and is taking such drastic measures without negotiating a payment plan or extending the moratorium on non-federal consumer debt collection levies and garnishments. The customer believes Fidelity's actions are egregiously wrong, predatory, and disrespectful to their financial situation.


llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    45 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    54 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   31946.66 ms /    99 tokens
Llama.generate: 64 prefix-match hit, remaining 424 prompt tokens to eval


A customer reported an incident of fraudulent activity where someone used their identity to submit a transaction without their consent. The individual allegedly obtained good services and had credit extended, but the customer was not contacted or personally verified for application information during daytime hours in the evening.


llama_perf_context_print:        load time =   87257.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   424 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   155 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  156962.63 ms /   579 tokens


 The user input describes a section of the Fair Credit Reporting Act (FCRA) regarding identity theft. When a consumer identifies potential identity theft, the credit reporting agency must block reporting information related to that transaction on the business day after receiving appropriate proof from the consumer. The furnisher must be notified and the blocking is effective immediately. If it's determined there was no error or material misrepresentation, the consumer may request rescission of the block. Consumers are promptly notified when information is reinstated. Resellers have obligations to provide notice if they file a report concerning the identified consumer. Check service companies must apply certain provisions and report to national credit reporting agencies. Law enforcement agencies cannot access blocked information without proper authorization or court order.


In [ ]:
gold_examples.head()

,product,narrative,summary,mistral_response
167,retail_banking,fraudulent charge totaling made capital one ch...,A fraudulent charge was made on the individual...,The customer reported a fraudulent charge of a...
169,credit_reporting,block except otherwise provided section consum...,The text outlines various stipulations regardi...,The customer complaint is about a consumer re...
461,credit_card,usaa master plan collect cancellation debt usa...,The input appears to be a complaint about USAA...,The customer is disputing two delinquent accou...
253,credit_reporting,block except otherwise provided section consum...,The text pertains to the stipulations and oper...,The customer complaint is about a consumer re...
42,credit_reporting,open account acct opened balance account acct ...,The input is about various accounts being open...,The user input consists of multiple repetition...


##### **Q4.3: Evaluate bert score** **(1 Marks)**

In [ ]:
def evaluate_score(test_data, scorer, bert_score=False):

    """
    Return the ROUGE score or BERTScore for predictions on gold examples
    For each example we make a prediction using the prompt.
    Gold summaries and the AI generated summaries are aggregated into lists.
    These lists are used by the corresponding scorers to compute metrics.
    Since BERTScore is computed for each candidate-reference pair, we take the
    average F1 score across the gold examples.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        scorer (function): Scorer function used to compute the ROUGE score or the
                           BERTScore
        bert_score (boolean): A flag variable that indicates if BERTScore should
                              be used as the metric.

    Output:
        score (float): BERTScore or ROUGE score computed by comparing model predictions
                       with ground truth
    """

    model_predictions = test_data['mistral_response'].tolist()
    ground_truths = test_data['summary'].tolist()
    if bert_score:
        score = scorer.compute(
            predictions=model_predictions,
            references=ground_truths,
            lang="en",
            rescale_with_baseline=True
        )

        return sum(score['f1'])/len(score['f1'])
    else:
        return scorer.compute(
            predictions=model_predictions,
            references=ground_truths
        )

In [ ]:
bert_scorer = evaluate.load("bertscore")

In [ ]:
score = evaluate_score(gold_examples,
    bert_scorer,
    bert_score=True)

print(f'BERTScore: {score}')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: 0.3459798875451088


In [4]:
import nbformat
from nbconvert import HTMLExporter

# Path to the input notebook file
input_notebook_path = "/content/drive/MyDrive/Colab Notebooks/learners-notebook-mid-term-project.ipynb"

# Path to save the output HTML file
output_html_path = "/content/drive/MyDrive/Colab Notebooks/learners-notebook-mid-term-project.html"

# Read the notebook content
with open(input_notebook_path, "r", encoding="utf-8") as f:
    notebook_content = nbformat.read(f, as_version=4)

# Remove widget state metadata
if "metadata" in notebook_content:
    notebook_content["metadata"].pop("widgets", None)

# Initialize the HTMLExporter
html_exporter = HTMLExporter()

# Convert the notebook to HTML
body, resources = html_exporter.from_notebook_node(notebook_content)

# Write the HTML output to a file
with open(output_html_path, "w", encoding="utf-8") as f:
    f.write(body)

print(f"Notebook successfully converted to HTML: {output_html_path}")

Notebook successfully converted to HTML: /content/drive/MyDrive/Colab Notebooks/learners-notebook-mid-term-project.html
